#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/competitions/desafio-de-prediccion-de-precios-de-portatiles

## Librerias

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ToolBox as TB
import re

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

## Data

In [53]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
df_train = pd.read_csv ('Data/train.csv')
df_test = pd.read_csv ('Data/test.csv')

In [54]:
print (df_train.shape)
print (df_test.shape)

(912, 14)
(391, 13)


## Data exploration

In [55]:
df_train.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                912 non-null    int64  
 1   laptop_ID         912 non-null    int64  
 2   Company           912 non-null    object 
 3   Product           912 non-null    object 
 4   TypeName          912 non-null    object 
 5   Inches            912 non-null    float64
 6   ScreenResolution  912 non-null    object 
 7   Cpu               912 non-null    object 
 8   Ram               912 non-null    object 
 9   Memory            912 non-null    object 
 10  Gpu               912 non-null    object 
 11  OpSys             912 non-null    object 
 12  Weight            912 non-null    object 
 13  Price_euros       912 non-null    float64
dtypes: float64(2), int64(2), object(10)
memory usage: 99.9+ KB


In [56]:
df_test.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                391 non-null    int64  
 1   laptop_ID         391 non-null    int64  
 2   Company           391 non-null    object 
 3   Product           391 non-null    object 
 4   TypeName          391 non-null    object 
 5   Inches            391 non-null    float64
 6   ScreenResolution  391 non-null    object 
 7   Cpu               391 non-null    object 
 8   Ram               391 non-null    object 
 9   Memory            391 non-null    object 
 10  Gpu               391 non-null    object 
 11  OpSys             391 non-null    object 
 12  Weight            391 non-null    object 
dtypes: float64(1), int64(2), object(10)
memory usage: 39.8+ KB


In [57]:
df_train.describe ()

,id,laptop_ID,Inches,Price_euros
count,912.000000,912.000000,912.000000,912.000000
mean,645.091009,666.192982,15.011404,1108.122873
std,380.313813,384.873846,1.411744,714.597741
min,0.000000,1.000000,10.100000,174.000000
25%,319.750000,330.500000,14.000000,589.000000
50%,636.500000,673.500000,15.600000,949.000000
75%,980.250000,998.500000,15.600000,1458.500000
max,1302.000000,1320.000000,18.400000,6099.000000


In [58]:
df_train.head ()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37


In [59]:
TB.describe_df (df_train)

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
DATA_TYPE,int64,int64,object,object,object,float64,object,object,object,object,object,object,object,float64
MISSINGS (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UNIQUE_VALUES,912,912,19,475,6,15,35,104,8,36,91,9,158,614
CARDIN (%),100.0,100.0,2.08,52.08,0.66,1.64,3.84,11.4,0.88,3.95,9.98,0.99,17.32,67.32


In [60]:
TB.tipifica_variables (df_train, 10, 30)

,nombre_variable,tipo_sugerido
0,id,Numérica Continua
1,laptop_ID,Numérica Continua
2,Company,Numérica Discreta
3,Product,Numérica Continua
4,TypeName,Categórica
5,Inches,Numérica Discreta
6,ScreenResolution,Numérica Discreta
7,Cpu,Numérica Discreta
8,Ram,Categórica
9,Memory,Numérica Discreta


In [61]:
df_train.head ()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37


## Data processing

In [62]:
print(df_test.columns)


Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight'],
      dtype='object')


In [63]:
# Columna Cpu

def extract_from_Cpu(train):
    train["Vel"] = train["Cpu"].str.extract(r'(\d+(?:\.\d+)?GHz)')
    train["Vel"] = train["Vel"].str.replace('GHz','')
    train["Vel"] = train["Vel"].astype(float)
    train["Cpu"] = train["Cpu"].str.replace(r'(\d+(?:\.\d+)?GHz)','',regex =True)
    

extract_from_Cpu (df_test)
extract_from_Cpu (df_train)


In [64]:
# Columna ScreenResolution
def extract_screen_features(train):
    #train = train.rename(columns = str.lower)
    train["resolucion"] = train['ScreenResolution'].str.extract(r'(\d+x\d+)')
    train["pantalla"]= train['ScreenResolution'].replace(r'(\d+x\d+)','', regex = True)
    train['pantalla'] = train['pantalla'].replace(r'(Full HD|Quad HD|Quad HD|\+|/|4K Ultra HD)','', regex = True)
    train['tactil'] = train['pantalla'].str.extract(r'(Touchscreen)')
    train['pantalla'] = train['pantalla'].replace(r'(Touchscreen)','', regex = True)
    train['tactil'] = train['tactil'].replace('Touchscreen', 1)
    train['tactil'] = train['tactil'].replace(np.nan, 0)
    train['pantalla'] = train['pantalla'].replace(r' ','', regex = True)
    train['pantalla'] = train['pantalla'].replace(r'^\s*$', np.nan, regex = True)
    train = train.drop("ScreenResolution",axis = 1)
    

extract_screen_features (df_test)
extract_screen_features (df_train)

C:\Users\pobla\AppData\Local\Temp\ipykernel_25076\1194225786.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['tactil'] = train['tactil'].replace('Touchscreen', 1)
C:\Users\pobla\AppData\Local\Temp\ipykernel_25076\1194225786.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['tactil'] = train['tactil'].replace('Touchscreen', 1)


In [65]:
def process_ram_column(dataframe):
    dataframe['Ram'] = dataframe['Ram'].str.replace('GB', '').astype(float)
    
process_ram_column (df_train)
process_ram_column (df_test)

In [66]:
def extract_from_storage(train):
    train['almacenamiento1'] = train['Memory']
    train['almacenamiento1'] = train['almacenamiento1'].str.replace('1.0TB','1TB', regex = True)
    train['almacenamiento1'] = train['almacenamiento1'].str.replace('1TB','1000GB')
    train['almacenamiento1'] = train['almacenamiento1'].str.replace('2TB','2000GB')
    train['almacenamiento1'] = train['almacenamiento1'].str.replace('GB','')
    train['almacenamiento2'] = train['almacenamiento1'].str.replace(r' ','')
    almacenamiento1 = []
    almacenamiento2 = []
    for i in train['almacenamiento2']:
        if len(re.findall(r'\+', i)) == 1: 
            # Double drive
            one = re.findall(r'(\w+)', i)
            almacenamiento1.append(one[0])
            almacenamiento2.append(one[1])
        else: 
            # Single drive
            one = re.findall(r'(\w+)', i)
            almacenamiento1.append(one[0])
            almacenamiento2.append('NaN')


    #extracting size and type of primary Memory
    storage1size = []
    storage1type = []
    for i in almacenamiento1:
        storage1type.append(re.findall(r'(\D\w+)', i)[0])
        storage1size.append(re.findall(r'(\d+)', i)[0])


    #extracting size and type of secondary Memory
    storage2size = []
    storage2type = []
    for i in almacenamiento2:
        if i != 'NaN':
            storage2type.append(re.findall(r'(\D\w+)',i)[0])
            storage2size.append(re.findall(r'(\d+)',i)[0])
        else:
            storage2type.append('NaN')
            storage2size.append(0)
    train['primarystorage_size'] = storage1size
    train['primarystorage_type'] = storage1type
    train['secondarystorage_size'] = storage2size
    train['secondarystorage_type'] = storage2type
    
    train["primarystorage_size"] = train["primarystorage_size"].astype(float)
    train["secondarystorage_size"] = train["secondarystorage_size"].astype(float)
    train = train.drop(['almacenamiento1','almacenamiento2','Memory'], axis = 1)
    train = train.replace({'NaN' : np.nan})
    


extract_from_storage (df_train)
extract_from_storage (df_test)

In [67]:
df_test ['Weight'].value_counts ()

Weight
2.2kg     35
2.1kg     18
2.5kg     14
2.4kg     13
2.8kg     13
          ..
2.33kg     1
2.65kg     1
1.22kg     1
3.25kg     1
1.23kg     1
Name: count, Length: 125, dtype: int64

In [68]:
def clean_weight(train):
    train['Peso'] = train['Weight'].str.replace('kg','')
    #train['Peso'] = train['Weight'].str.replace('s', '').astype(float)
    
clean_weight (df_train)
clean_weight (df_test)

In [69]:
def manufacturer(train):
    train['cpu_manftr'] = train['Cpu'].str.extract(r'^(\w+)') 
    train['gpu_manftr'] = train['Gpu'].str.extract(r'^(\w+)') 
    

manufacturer (df_test)
manufacturer (df_train)

In [70]:
df_test.columns

Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'Vel', 'resolucion', 'pantalla', 'tactil', 'almacenamiento1',
       'almacenamiento2', 'primarystorage_size', 'primarystorage_type',
       'secondarystorage_size', 'secondarystorage_type', 'Peso', 'cpu_manftr',
       'gpu_manftr'],
      dtype='object')

In [75]:
df_train.head ()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,...,tactil,almacenamiento1,almacenamiento2,primarystorage_size,primarystorage_type,secondarystorage_size,secondarystorage_type,Peso,cpu_manftr,gpu_manftr
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U,8.0,1TB HDD,...,0.0,1000 HDD,1000HDD,1000.0,HDD,0.0,NaN,2.36,Intel,AMD
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200,4.0,1TB HDD,...,0.0,1000 HDD,1000HDD,1000.0,HDD,0.0,NaN,2,Intel,Intel
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U,8.0,256GB SSD,...,0.0,256 SSD,256SSD,256.0,SSD,0.0,NaN,1.2,Intel,Intel
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ,16.0,256GB SSD + 1TB HDD,...,0.0,256 SSD + 1000 HDD,256SSD+1000HDD,256.0,SSD,1000.0,HDD,4.42,Intel,Nvidia
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U,8.0,256GB SSD,...,1.0,256 SSD,256SSD,256.0,SSD,0.0,NaN,1.26,Intel,Intel


-----------------------------------------------------------------------------------------------------------------

## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

### 2. Dividir X_train, X_test, y_train, y_test

### 3. Crear y entrenar el modelo


### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de MAE.

-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 2. Carga los datos de `test.csv` para predecir.

**¿Por qué puede dar error?** 

IMPORTANTE: APLICAR A ESTOS DATOS LO MISMO QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [72]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [73]:
predictions = model.predict(X_pred)
predictions

NameError: name 'model' is not defined

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [ ]:
sample = pd.read_csv("Dataset/sample_submission.csv") # Esta es mi ruta del archivo, la vuestra puede ser otra

In [ ]:
sample.head()

In [ ]:
sample.shape

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [ ]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("")


In [ ]:
chequeator(submission) # submission es el nombre que le habríamos puesto a nuestro .csv con los valores que me salieron en la predicción

You're ready to submit!
